# ResNet Detector Developement 1

WNixalo - 1:7/5/2018

---

The purpose of this notebook is to experiment with small datasets (CPU dev) and finetune the general-purpose base object detector on the GLoC dataset.

## Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *

sys.path.insert(1, os.path.join('../'))
from utils import common
from utils import temp_utils
from utils.subfolder_val_idxs import set_val_idxs

from matplotlib import patches, patheffects

In [3]:
PATH = Path('../data')
PATH_IMAGES   = PATH/'train'
PATH_CLAS_CSV = PATH/'class_labels.csv'
PATH_BBOX_CSV = PATH/'bbox_labels.csv'

# PATH_TRAIN_BBX = PATH/'interstage_train'
# CPU_PATH_CSV     = PATH/'cpu_labels.csv'
# CPU_PATH_CSV_BBX = PATH/'cpu_interstage_labels.csv'

## 1. Data

In [4]:
df = pd.read_csv(PATH_BBOX_CSV)
df.head()

,id,bbox
0,000000-000412/000000.jpg,72 83 380 191
1,000000-000412/000001.jpg,89 52 381 204
2,000000-000412/000002.jpg,89 58 390 208
3,000000-000412/000003.jpg,98 66 388 214
4,000000-000412/000004.jpg,90 65 389 209


### CPU data subset - TODO

***NOTE***: I'll have to rewrite my validation set indexer to work with CSVs instead of directories.

In [4]:
# TODO

### Visualizations - TODO

In [93]:
bbx = cpu_bbx_df.iloc[0].values[1:5]; bbx

array([148, 130, 246, 386], dtype=object)

In [379]:
def show_img(img, bbx, figsize=None, ax=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(img)
    ax.get_xaxis().set_visible(False); ax.get_yaxis().set_visible(False)
    return ax

def bb_hw(bbx):
    """returns (h w) from  (x1 y1 x2 y2) bounding box"""
    h = bbx[3]-bbx[1]
    w = bbx[2]-bbx[0]
    return h,w

def draw_outline(o, λw):
    o.set_path_effects([patheffects.Stroke(linewidth=λw, foreground='black'), patheffects.Normal()])

def draw_rect(ax, bbx):
    # draw bounding box
    h,w = bb_hw(bbx)
    patch = ax.add_patch(patches.Rectangle(bbx[:2], w,h, fill=False, edgecolor='white', lw=2)) # *bbx[:2] destructs bbx giving bbx[-2], bbx[-1]
    # outline it
    draw_outline(patch, 4)

def draw_text(ax, xy, txt, sz=14):
    # draw classification
    text = ax.text(*xy, 'Pilot', verticalalignment='top', color='white', fontsize=sz, weight='bold')
    # outline it
    draw_outline(text, 1)
    
def display_anno(img, bbx, clsfn=None):
    ax = show_img(img, bbx)
    draw_rect(ax, bbx)
    draw_text(ax, bbx[:2], clsfn)

### fastai ModelData Object

In [66]:
f_model = resnet34
sz = 224
bs = 32

In [67]:
aug_tfms = [RandomRotate(3, p=0.5, tfm_y=TfmType.COORD), 
            RandomLighting(0.05, 0.05, tfm_y=TfmType.COORD), 
            RandomFlip(tfm_y=TfmType.COORD)]

tfms = tfms_from_model(f_model, sz, crop_type=CropType.NO, tfm_y=TfmType.COORD, aug_tfms=aug_tfms)
model_data = ImageClassifierData.from_csv(PATH, PATH_IMAGES, PATH_BBOX_CSV, tfms=tfms, continuous=True)

#### Adding Class labels to Coordinate labels

In [68]:
class ConcatLblDataset(Dataset):
    def __init__(self, ds, y2):
        self.ds,self.y2 = ds, y2
        self.sz = ds.sz
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, i):
        x,y = self.ds[i]
        return (x, (y, self.y2[i]))

In [69]:
len(model_data.val_dl.dataset.fnames)

1527

In [70]:
len(model_data.trn_dl.dataset.fnames)

6110

In [71]:
1527 / 6110

0.24991816693944355

Wow. The fastai library automatically sets a random validation set of 25% of your data if you don't specify any. Cool.

In [72]:
df_mcs = pd.read_csv(PATH_CLAS_CSV)

multi_classes = np.array([np.array([clasid]) for clasid in df_mcs['class']])

trn2 = ConcatLblDataset(model_data.trn_ds, multi_classes)

In [75]:
model_data.trn_dl.dataset = trn2

In [77]:
model_data.trn_dl.dataset[0]

(array([[[-1.80786, -1.80786, -1.80786, ..., -1.80786, -1.80786, -1.80786],
         [-1.97226, -1.97226, -1.97226, ..., -1.97226, -1.97226, -1.97226],
         [-1.97226, -1.97226, -1.97226, ..., -1.97226, -1.97226, -1.97226],
         ...,
         [-1.97226, -1.97226, -1.97226, ..., -1.97226, -1.97226, -1.97226],
         [-1.97226, -1.97226, -1.97226, ..., -1.97226, -1.97226, -1.97226],
         [-1.97226, -1.97226, -1.97226, ..., -1.97226, -1.97226, -1.97226]],
 
        [[-1.71875, -1.71875, -1.71875, ..., -1.71875, -1.71875, -1.71875],
         [-1.88682, -1.88682, -1.88682, ..., -1.88682, -1.88682, -1.88682],
         [-1.88682, -1.88682, -1.88682, ..., -1.88682, -1.88682, -1.88682],
         ...,
         [-1.88682, -1.88682, -1.88682, ..., -1.88682, -1.88682, -1.88682],
         [-1.88682, -1.88682, -1.88682, ..., -1.88682, -1.88682, -1.88682],
         [-1.88682, -1.88682, -1.88682, ..., -1.88682, -1.88682, -1.88682]],
 
        [[-1.48889, -1.48889, -1.48889, ..., -1.48889,

## 2. Architecture

In [ ]:
class ConvBlock

In [ ]:
class SSD_Head(nn.Module):
    ## TODO

## 3. Loss Function

## 4. Training